# Import

pip install matplotlib requests openai pandas geopandas numpy mapclassify langchain pydantic huggingface_hub transformers==4.30 geopy geocoder accelerate bitsandbytes einops fuzzywuzzy xformers unidecode inflect nlpaug dill osmium

In [72]:
import pandas as pd
import requests
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from tqdm import tqdm

# Load data

In [79]:
ticker_to_stock = {'GOOG': 'Alphabet Inc. (Google, GOOG)', 
                   'V': 'Visa (V)', 
                   'BRK': 'Berkshire Hathaway (BRK)', 
                   'GE': 'General Electric (GE)', 
                   'XOM': 'ExonMobil (XOM)'}

In [80]:
ext_df = pd.read_csv('/code/data/extracted_data.csv')
ext_df.head()

,Date,Article_title,Stock_symbol,Url,Publisher,Author,Article,Lsa_summary,Luhn_summary,Textrank_summary,Lexrank_summary
0,2023-12-16 19:00:00 UTC,Guru Fundamental Report for V,V,https://www.nasdaq.com/articles/guru-fundament...,NaN,NaN,Below is Validea's guru fundamental report for...,VISA INC (V) is a large-cap growth stock in th...,"Of the 22 guru strategies we follow, V rates h...",Detailed Analysis of VISA INC V Guru Analysis ...,Below is Validea's guru fundamental report for...
1,2023-12-16 10:00:00 UTC,"Notable Monday Option Activity: CVX, SNDR, V",V,https://www.nasdaq.com/articles/notable-monday...,NaN,NaN,Looking at options trading activity among comp...,That number works out to 42.8% of CVX's averag...,Particularly high volume was seen for the $152...,Below is a chart showing CVX's trailing twelve...,Below is a chart showing SNDR's trailing twelv...
2,2023-12-16 05:00:00 UTC,Visa (V) to Expand Footprint in Mexico With Pr...,V,https://www.nasdaq.com/articles/visa-v-to-expa...,NaN,NaN,Visa Inc. V recently announced a definitive ag...,This move will help Visa promote digitization ...,Image Source: Zacks Investment Research Stocks...,Image Source: Zacks Investment Research Stocks...,Prosa will be able to offer new benefits to ca...
3,2023-12-16 00:00:00 UTC,"GS, MA, V: Top Hedge Fund Expert Favors These ...",V,https://www.nasdaq.com/articles/gs-ma-v%3A-top...,NaN,NaN,Given the importance of the financial sector i...,TipRanks simplifies this process by offering t...,Given the importance of the financial sector i...,Given the importance of the financial sector i...,TipRanks simplifies this process by offering t...
4,2023-12-15 00:00:00 UTC,Visa (V) Enhances Offerings With Visa Provisio...,V,https://www.nasdaq.com/articles/visa-v-enhance...,NaN,NaN,Visa Inc. V recently launched Visa Provisionin...,Visa has invested more than $10 billion in the...,Image Source: Zacks Investment Research Stocks...,Image Source: Zacks Investment Research Stocks...,Visa aims to secure the transactions of Visa c...


In [81]:
test_ticker = ext_df.iloc[1].Stock_symbol
test_article = ext_df.iloc[1].Article
print(test_ticker, '\n\n', test_article)

V 

 Looking at options trading activity among components of the Russell 3000 index, there is noteworthy activity today in Chevron Corporation (Symbol: CVX), where a total volume of 51,833 contracts has been traded thus far today, a contract volume which is representative of approximately 5.2 million underlying shares (given that every 1 contract represents 100 underlying shares). That number works out to 42.8% of CVX's average daily trading volume over the past month, of 12.1 million shares. Particularly high volume was seen for the $152.50 strike call option expiring December 22, 2023, with 5,762 contracts trading so far today, representing approximately 576,200 underlying shares of CVX. Below is a chart showing CVX's trailing twelve month trading history, with the $152.50 strike highlighted in orange:
Schneider National Inc (Symbol: SNDR) saw options trading volume of 2,583 contracts, representing approximately 258,300 underlying shares or approximately 42.6% of SNDR's average daily

# Stuff

## Test

In [82]:
def clean_output(output):
    output = output[:2]
    output = output.strip(':`.!?\n\\#')
    return output

In [83]:
llama2_prompt_template = """### System: 
Forget all your previous instructions. You are a financial expert with stock recommendation experience. News articles will be passed in each time, you will give only a single score between 1-5 as an answer. Based on a specific stock, score what sentiment the article implies on a certain stock: range from 1 to 5, where 1 is negative, 2 is somewhat negative, 3 is neutral, 4 is somewhat positive, 5 is positive.
Examples:
 `Apple got better earnings than expected`: Answer: `5` 
 `Apple got bankrupt`: Answer: `1` 

### Article:
```
{article}
```

What is the implied sentiment of the article to {stock} stock? Answer only a single number between 1 and 5.

### Assistant:
The implied sentiment is """

In [88]:
headers = {"Content-Type": "application/json"}
def get_sentiment(article, stock):
    template = llama2_prompt_template
    prompt = template.format(article=article, stock=stock)
    
    json_data = {"prompt": prompt, "max_tokens": 1, "temperature": 0.01,}
    response = requests.post('http://oobabooga:5000/api/v1/generate', headers=headers, json=json_data).json()['results'][0]['text']
    response =  clean_output(response)
    return response

get_sentiment(ext_df.iloc[10].Article, ticker_to_stock[ext_df.iloc[10].Stock_symbol])

'5'

## Predict sentiment of articles

In [90]:
model_name = 'TheBloke_Llama-2-13B-GPTQ'

sentiments = []
get_token = lambda x: len(x.split(' '))/0.7

for idx, row in tqdm(ext_df.iterrows(), total=len(ext_df)):
    ticker = row.Stock_symbol
    article = row.Article
    if get_token(test_article) > 2000:
        continue
    
    sentiment = get_sentiment(article, ticker)
    sentiments.append({'Stock_symbol': ticker, 'Date': row.Date, 'Url': row.Url, 'Sentiment': sentiment})
    
    if (idx % 500) == 0:
        pd.DataFrame(sentiments).to_csv(f'/code/data/{model_name}.csv', index=False)

pd.DataFrame(sentiments).to_csv(f'/code/data/{model_name}.csv', index=False)
sentiments

  1%|▏         | 104/6945 [05:35<6:08:18,  3.23s/it] 


KeyboardInterrupt: 